# K-nn algorithm using SVD pre-processing

As we train our k-nn model on vectors of with a really large dimension, we would want to reduce the dimension and will thus use SVD : Singular Values Decomposition in order to project our vectors in a smaller space, and use cosine similarity as a distance between vectors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

Let's start by testing how our dataset responds to SVD

In [8]:
#Get the data

#read data
data = pd.read_csv('steam-200k.csv')
#clean data
data.columns = ['id','game','state','hours_played','0']
data = data.drop('0',axis=1)
played_games = data.loc[data['state']=='play']
#Get a dict of games and hours played for each id
played_dict = played_games.groupby('id').apply(lambda g : dict(zip(g['game'], g['hours_played'])))

In [50]:
#Standardize
standardization_dict = dict()
for game_name, s in played_games.groupby('game')['hours_played']:
    standardization_dict[game_name]=dict()
    series = s[s>0.0]  #take only games played
    standardization_dict[game_name]['average'] = series.mean()
    if series.std() > 0 : #for some games that are not much played, std is 0 which creates errors
        standardization_dict[game_name]['std'] = series.std()
    else:
        standardization_dict[game_name]['std'] = 1e-8

def standardize(game, hours):
    return (hours - standardization_dict[game]['average'])/standardization_dict[game]['std']

standardized = played_games.copy()

standardized['hours_played'] = standardized.apply(lambda x : standardize(x.game, x.hours_played),axis=1)


To apply SVD, we want to project our players in a smaller space depending on the games they played, so want the players in columns and the games in rows

In [34]:
svd_encoding = standardized.drop_duplicates(subset=['game','id'])
svd_encoding = svd_encoding.pivot(index='game', columns='id', values='hours_played')
svd_encoding = svd_encoding.fillna(0)

svd_encoding.head()

id,5250,76767,86540,144736,181212,229911,298950,381543,547685,554278,...,309228590,309255941,309262440,309265377,309404240,309434439,309554670,309626088,309824202,309903146
game,,,,,,,,,,,,,,,,,,,,,
007 Legends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Second Ninja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
u, s, vh = np.linalg.svd(svd_encoding)

In [36]:
k = 5 #number of components kept
uk = u[:,:k]
sk = np.diag(s[:k])
vhk = vh[:k,:]

hours_KD = svd_encoding.T@uk

In [37]:
hours_KD = pd.DataFrame(hours_KD)
hours_KD

,0,1,2,3,4
id,,,,,
5250,-0.338704,-0.241455,0.018510,-0.180361,0.013846
76767,-0.004468,-0.045326,0.625901,0.118420,-0.108568
86540,-0.000450,-0.021970,-0.042716,0.213445,-0.019004
144736,0.000483,-0.000141,0.009455,0.002186,-0.000771
181212,0.000752,0.001373,-0.000305,0.068564,-0.009787
...,...,...,...,...,...
309434439,-0.335904,0.000417,-0.003479,0.002401,0.000676
309554670,-0.000066,0.000018,0.000796,0.000232,0.000218
309626088,0.000569,-0.004437,-0.002730,0.023579,-0.002980


Let's try to find what the components explain !

In [44]:
#Outlier of component 0
played_dict[hours_KD.iloc[:,0].idxmax()]

{'Dota 2': 10442.0,
 'Counter-Strike Global Offensive': 405.0,
 'Rust': 3.4,
 'FreeStyle2 Street Basketball': 2.5,
 'Aura Kingdom': 0.2,
 'Left 4 Dead 2': 0.1}

In [45]:
#Outlier of component 1
played_dict[hours_KD.iloc[:,1].idxmax()]

{'Team Fortress 2': 9640.0}

In [46]:
#Outlier of component 2
played_dict[hours_KD.iloc[:,2].idxmax()]

{'The Elder Scrolls V Skyrim': 1090.0,
 'Mount & Blade Warband': 825.0,
 'Dragon Age Origins': 405.0,
 'Chivalry Medieval Warfare': 340.0,
 'Saints Row The Third': 333.0,
 'Terraria': 327.0,
 'Counter-Strike Source': 316.0,
 'Mass Effect 2': 311.0,
 'Saints Row 2': 308.0,
 'Dota 2': 235.0,
 'Far Cry 3': 234.0,
 'Age of Empires II HD Edition': 227.0,
 'Just Cause 2': 196.0,
 "Assassin's Creed IV Black Flag": 185.0,
 "Assassin's Creed II": 182.0,
 'Portal 2': 178.0,
 'Tropico 4': 170.0,
 'The Witcher 3 Wild Hunt': 154.0,
 'Deus Ex Human Revolution': 147.0,
 'Fallout New Vegas': 139.0,
 'Sleeping Dogs': 139.0,
 'Grand Theft Auto San Andreas': 135.0,
 'BioShock Infinite': 135.0,
 'Max Payne 3': 126.0,
 'Need for Speed Hot Pursuit': 122.0,
 'Wolfenstein The New Order': 122.0,
 'The Elder Scrolls III Morrowind': 116.0,
 'Far Cry 3 Blood Dragon': 107.0,
 'Grand Theft Auto V': 103.0,
 'Worms Reloaded': 91.0,
 'From Dust': 91.0,
 'Middle-earth Shadow of Mordor': 89.0,
 "Assassin's Creed Revelat

In [47]:
#Outlier of component 3
played_dict[hours_KD.iloc[:,3].idxmax()]

{'Counter-Strike Global Offensive': 663.0,
 "Sid Meier's Civilization V": 550.0,
 'Total War SHOGUN 2': 212.0,
 'Total War ROME II - Emperor Edition': 198.0,
 'Dungeon Defenders': 195.0,
 'Age of Empires Online': 168.0,
 'XCOM Enemy Unknown': 126.0,
 'Empire Total War': 125.0,
 'Might & Magic Heroes VI': 118.0,
 "Assassin's Creed IV Black Flag": 94.0,
 'Alien Swarm': 83.0,
 "Assassin's Creed II": 79.0,
 "Assassin's Creed Brotherhood": 73.0,
 'Terra Incognita ~ Chapter One The Descendant': 61.0,
 'Warlock - Master of the Arcane': 55.0,
 'Phantom Breaker Battle Grounds': 53.0,
 'Soul Gambler': 48.0,
 'Supreme Commander': 47.0,
 'The Incredible Adventures of Van Helsing II': 43.0,
 'Titan Quest Immortal Throne': 40.0,
 'Orcs Must Die! 2': 39.0,
 "Assassin's Creed Revelations": 38.0,
 'Relic Hunters Zero': 38.0,
 "Tom Clancy's Ghost Recon Advanced Warfighter": 37.0,
 'Warhammer 40,000 Dawn of War II - Chaos Rising': 35.0,
 'Napoleon Total War': 34.0,
 'Warhammer 40,000 Dawn of War II  Retr

In [48]:
#Outlier of component 4
played_dict[hours_KD.iloc[:,4].idxmax()]

{'The Elder Scrolls V Skyrim': 1090.0,
 'Mount & Blade Warband': 825.0,
 'Dragon Age Origins': 405.0,
 'Chivalry Medieval Warfare': 340.0,
 'Saints Row The Third': 333.0,
 'Terraria': 327.0,
 'Counter-Strike Source': 316.0,
 'Mass Effect 2': 311.0,
 'Saints Row 2': 308.0,
 'Dota 2': 235.0,
 'Far Cry 3': 234.0,
 'Age of Empires II HD Edition': 227.0,
 'Just Cause 2': 196.0,
 "Assassin's Creed IV Black Flag": 185.0,
 "Assassin's Creed II": 182.0,
 'Portal 2': 178.0,
 'Tropico 4': 170.0,
 'The Witcher 3 Wild Hunt': 154.0,
 'Deus Ex Human Revolution': 147.0,
 'Fallout New Vegas': 139.0,
 'Sleeping Dogs': 139.0,
 'Grand Theft Auto San Andreas': 135.0,
 'BioShock Infinite': 135.0,
 'Max Payne 3': 126.0,
 'Need for Speed Hot Pursuit': 122.0,
 'Wolfenstein The New Order': 122.0,
 'The Elder Scrolls III Morrowind': 116.0,
 'Far Cry 3 Blood Dragon': 107.0,
 'Grand Theft Auto V': 103.0,
 'Worms Reloaded': 91.0,
 'From Dust': 91.0,
 'Middle-earth Shadow of Mordor': 89.0,
 "Assassin's Creed Revelat

The different components seem to represent extremely accurately the kind of players in the dataset.  
Indeed, a huge part of the players only plays 3 popular games : Dota 2, Team Fortress 2 and Counter-Strike Global offensive.  
The two first components enables to separate these players, and the next ones represent players that like other types of games.  
3rd component seems to represent FPS/strategy games players, and 4th and 5th one represent RPGs players which are two huge types of players.  
This methods thus seems to work impressively well while necessiting only a few calculations.  
Let's try it as an input to our model !

To do so, we first need to redefine a distance metrics, and we will choose cosine similarity as it is the most common metrics in this case.  
To be precise, cosine similarity is not a distance in a mathematical sense, but it is not a problem for K-nn.  

Let's implement the svd projection in the fitting part of our model, and using cosine similarity for predicting neighbors.  

To be able to project the vector for prediction in the space, we will store the uk matrix.

In [ ]:
class SteamPredictionModel():
    
    def __init__(self, k_neighbors = 5):
        self.neigh = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean')
        self.games_list = []
        self.likeness = None
        self.mean_dict = {}
        self.std_dict = {}
        self.average_played = {}
        self.uk_matrix = None

    def dict_to_likeness(self, dicti):
        d = dicti.copy()
        for game in d.keys():
            if d[game] <= self.average_played[game]:
                d[game]=0
        return d
    
    #We fit the model on a dataset containing ids and dictionnaries of games associated with time played
    def fit(self, data):

        #Firstly we encode the hours played
        hours_encoded = data.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        hours_encoded = hours_encoded.fillna(0)
        hours_encoded = hours_encoded.reindex(sorted(hours_encoded.columns),axis=1)
        

        #For each player, we compute the list of game he likes with the time he has played aboved average time played
        non_zero_dict = hours_encoded.replace(0, np.NaN)
        self.average_played = non_zero_dict.mean(axis=0)
        likeness_games = data.map(self.dict_to_likeness)
        #And encode them
        likeness_games_encoded = likeness_games.apply(pd.Series)
        #Replace NaN values by 0 : a game not in the dict has never been played
        likeness_games_encoded = likeness_games_encoded.fillna(0)
        likeness_games_encoded = likeness_games_encoded.reindex(sorted(likeness_games_encoded.columns),axis=1)

        #standardization
        #We standardize each column separately

        def standardize(c):
            m = c.mean()
            if c.std() > 0:
                std = c.std()
            else:
                std = 1e-8
            return (c-m)/std

        hours_encoded = hours_encoded.apply(lambda column : standardize(column),axis=0)

        #we store the mean and std to standardize vectors on which we want to make predictions
        self.mean_dict = hours_encoded.mean(axis=0)
        self.std_dict = hours_encoded.std(axis=0)
        self.std_dict.replace(to_replace=0,value=1e-8,inplace=True)

        #we also standardize the likeness
        likeness_games_encoded = likeness_games_encoded.apply(lambda column : standardize(column),axis=0)
        self.likeness = likeness_games_encoded



        self.neigh.fit(hours_encoded.values)
        
        #Get the list of games
        games_list = list(hours_encoded.columns)
        games_list.sort()
        self.games_list = games_list
        
        
    
    #We predict a certain number of games (maximum) using a dictionnary of games associated with time played
    def predict(self, X_init, recommendations_number_max):
        #One-hot-encode X
        X = pd.Series(X_init,index=self.games_list).fillna(0)
        
        #Create a vector with all games and a null score
        score = pd.Series({self.games_list[0]:0.0},index=self.games_list).fillna(0)

        #Get the list of games played by X
        already_owned = [self.games_list[index] for index in np.asarray(X).nonzero()[0]]

        
        stand_vector = (X-self.mean_dict)/self.std_dict
        #Get the neighbors of X
        kneighbors = self.neigh.kneighbors([stand_vector])
        kneighbors_distances = kneighbors[0][0]
        kneighbors_indices = kneighbors[1][0]

        for i in range(len(kneighbors_indices)):
            neighbor = kneighbors_indices[i]
            #get the list of liked games
            liked = [self.games_list[index] for index in np.asarray(self.likeness.iloc[neighbor]).nonzero()[0]]
            #Add to each game score (1/d)*l with d the distance between X and the neighbor
            #and l the amount of time played above the average
            for liked_game in liked:
                if liked_game not in already_owned:
                    score[liked_game] = score[liked_game] + 1/kneighbors_distances[i] + self.likeness.iloc[neighbor][liked_game]


        score = score.sort_values(ascending=False)
        return score.iloc[:recommendations_number_max]